In [1]:
import sys
from collections import Counter

# Environment
copy paste from Eger

In [8]:
class Game(object):
    '''
    @ Eger
    '''
    def __init__(self, players, log=sys.stdout, format=0):
        self.players = players
        self.hits = 3  # num. life left
        self.hints = 8  # num. of hint tokens
        self.current_player = 0  # index for current player
        self.board = map(lambda c: (c,0), ALL_COLORS)  # no cards played at start
        self.played = []  # sucessfully played
        self.deck = make_deck()
        self.extra_turns = 0
        self.hands = []
        self.knowledge = []
        self.make_hands() 
        self.trash = []  # discarded + falsely played
        self.log = log
        self.turn = 1
        self.format = format
        self.dopostsurvey = False
        self.study = False
        if self.format:
            print >> self.log, self.deck

    # initial deck
    def make_hands(self):
        handsize = 4
        if len(self.players) < 4:
            handsize = 5
        for i, p in enumerate(self.players):
            self.hands.append([])
            self.knowledge.append([])
            for j in xrange(handsize):
                self.draw_card(i)
    def draw_card(self, pnr=None):
        if pnr is None:
            pnr = self.current_player
        if not self.deck:
            return
        self.hands[pnr].append(self.deck[0])
        self.knowledge[pnr].append(initial_knowledge())
        del self.deck[0]

    
    def perform(self, action):
        '''
        update knowledge and game board accordingly after an action
        '''
        for p in self.players:
            p.inform(action, self.current_player, self)
        if format:
            print >> self.log, "MOVE:", self.current_player, action.type, action.cnr, action.pnr, action.col, action.num
        if action.type == HINT_COLOR:
            self.hints -= 1
            print >>self.log, self.players[self.current_player].name, "hints", self.players[action.pnr].name, "about all their", COLORNAMES[action.col], "cards", "hints remaining:", self.hints
            print >>self.log, self.players[action.pnr].name, "has", format_hand(self.hands[action.pnr])
            for (col,num),knowledge in zip(self.hands[action.pnr],self.knowledge[action.pnr]):
                if col == action.col:
                    for i, k in enumerate(knowledge):
                        if i != col:
                            for i in xrange(len(k)):
                                k[i] = 0
                else:
                    for i in xrange(len(knowledge[action.col])):
                        knowledge[action.col][i] = 0
        elif action.type == HINT_NUMBER:
            self.hints -= 1
            print >>self.log, self.players[self.current_player].name, "hints", self.players[action.pnr].name, "about all their", action.num, "hints remaining:", self.hints
            print >>self.log, self.players[action.pnr].name, "has", format_hand(self.hands[action.pnr])
            for (col,num),knowledge in zip(self.hands[action.pnr],self.knowledge[action.pnr]):
                if num == action.num:
                    for k in knowledge:
                        for i in xrange(len(COUNTS)):
                            if i+1 != num:
                                k[i] = 0
                else:
                    for k in knowledge:
                        k[action.num-1] = 0
        elif action.type == PLAY:
            (col,num) = self.hands[self.current_player][action.cnr]
            print >>self.log, self.players[self.current_player].name, "plays", format_card((col,num)),
            if self.board[col][1] == num-1:
                self.board[col] = (col,num)
                self.played.append((col,num))
                if num == 5:
                    self.hints += 1
                    self.hints = min(self.hints, 8)
                print >>self.log, "successfully! Board is now", format_hand(self.board)
            else:
                self.trash.append((col,num))
                self.hits -= 1
                print >>self.log, "and fails. Board was", format_hand(self.board)
            del self.hands[self.current_player][action.cnr]
            del self.knowledge[self.current_player][action.cnr]
            self.draw_card()
            print >>self.log, self.players[self.current_player].name, "now has", format_hand(self.hands[self.current_player])
        else:
            self.hints += 1 
            self.hints = min(self.hints, 8)
            self.trash.append(self.hands[self.current_player][action.cnr])
            print >>self.log, self.players[self.current_player].name, "discards", format_card(self.hands[self.current_player][action.cnr])
            print >>self.log, "trash is now", format_hand(self.trash)
            del self.hands[self.current_player][action.cnr]
            del self.knowledge[self.current_player][action.cnr]
            self.draw_card()
            print >>self.log, self.players[self.current_player].name, "now has", format_hand(self.hands[self.current_player])
    
    
    def valid_actions(self):
        valid = []
        for i in xrange(len(self.hands[self.current_player])):
            valid.append(Action(PLAY, cnr=i))
            valid.append(Action(DISCARD, cnr=i))
        if self.hints > 0:
            for i, p in enumerate(self.players):
                if i != self.current_player:
                    for col in set(map(lambda p: p[0], self.hands[i])):  # p = (col,num)
                        valid.append(Action(HINT_COLOR, pnr=i, col=col))
                    for num in set(map(lambda p: p[1], self.hands[i])):
                        valid.append(Action(HINT_NUMBER, pnr=i, num=num))
        return valid
    def run(self, turns=-1):
        self.turn = 1
        while not self.done() and (turns < 0 or self.turn < turns):
            self.turn += 1
            if not self.deck:
                self.extra_turns += 1
            hands = []
            for i, h in enumerate(self.hands):
                if i == self.current_player:
                    hands.append([])
                else:
                    hands.append(h)
            action = self.players[self.current_player].get_action(self.current_player, hands, self.knowledge, self.trash, self.played, self.board, self.valid_actions(), self.hints)
            self.perform(action)
            self.current_player += 1
            self.current_player %= len(self.players)
        print >>self.log, "Game done, hits left:", self.hits
        points = self.score()
        print >>self.log, "Points:", points
        return points
    def score(self):
        return sum(map(lambda p: p[1], self.board))
    def single_turn(self):
        if not self.done():
            if not self.deck:
                self.extra_turns += 1
            hands = []
            for i, h in enumerate(self.hands):
                if i == self.current_player:
                    hands.append([])
                else:
                    hands.append(h)
            action = self.players[self.current_player].get_action(self.current_player, hands, self.knowledge, self.trash, self.played, self.board, self.valid_actions(), self.hints)
            self.perform(action)
            self.current_player += 1
            self.current_player %= len(self.players)
    def external_turn(self, action): 
        if not self.done():
            if not self.deck:
                self.extra_turns += 1
            self.perform(action)
            self.current_player += 1
            self.current_player %= len(self.players)
    def done(self):
        if self.extra_turns == len(self.players) or self.hits == 0:
            return True
        for (col,num) in self.board:
            if num != 5:
                return False
        return True
    def finish(self):
        if self.format:
            print >> self.log, "Score", self.score()
            self.log.close()

# New stuff

In [8]:
def pretend_action(self, postaction):
    '''
    Method for Class Game()
    simulate what will happen if the card-indicated agent perform certain action and compute score difference
    '''
    # archive the real game state
    c_knowledge = copy.deepcopy(knowledge)
    c_trash = copy.deepcopy(trash)
    c_played = copy.deepcopy(played)
    c_board = copy.deepcopy(board)
    c_hints = copy.deepcopy(hints)
    c_hits = copy.deepcopy(hits)
    
    # simualte performing the action and see how it changes the score
    if (postaction != PLAY) and (postaction != DISCARD):
        pass  # I guess nothing happens if one keeps a card
    else:
        perform(postaction)
    new_score = eval_score(hints, c_hints, hits, played, c_played, trash, c_trash, c_board, postaction)
    
    # bring back to the real game state
    knowledge = copy.deepcopy(c_knowledge)
    trash = copy.deepcopy(c_trash)
    played = copy.deepcopy(c_played)
    board = copy.deepcopy(c_board)
    hints = copy.deepcopy(c_hints)
    hits = copy.deepcopy(c_hits)
    
    
    return new_score - prev_score


def eval_score(hints, c_hints, hits, played, c_played, trash, c_trash, c_board, postaction):
    # init
    score = 0
    
    # hint token number
    score += -0.2 * (c_hints - hints)
    
    # number of error
    if hits == 3:
        score += 0
    elif hits == 2:
        score += -0.3
    elif hits ==1:
        score += -1.0
    else:
        score += -25
    
    # reward playing
    if len(played) - len(c_played) == 1:
        score += 1
    
    # discarding a card
    if len(trash) - len(c_trash)  == 1:
        trashed_card = set(trash) ^ set(c_trash)
        # discard a playable
        if playable(trashed_card, c_board): # if the trashed card was playable
            score += -0.5 
    
    # if action is KEEP
    if (postaction != PLAY) and (postaction != DISCARD):
        # TODO: KEEP Saving a card is rewarded
        
        # TODO: KEEP Blocking a card is bad, use self.keeplist from each agent?
        # This requires implementing keeplist as a var. of Class Game, a.t.m. it's a var. of Class Player
    
    
    # loosing a unique combination is bad
    for combi,occur in Count(trash).items():  # dict with a (col,rank):occurence pair
        if not discardable(combi,board):  # it hurts only when card was still useful
            if (combi[1]==0) and (occur==3):  # rank is 0-based so this is rank 1
                score += -(5-combi[1])  # -5
            elif (combi[1]==4) and (occur==1):  # rank 5 is available only once
                score += -(5-combi[1])  # -1
            elif (combi[1]!=0) and (occur==2):  # other ranks
                score += -(5-combi[1])
            elif (occur==2) or (occur==1):  # loosing a combi with spare available is not so bad
                score += -0.5
            else:
                pass

    # having a small stack is bad
    if ((postaction==PLAY) and (len(played) - len(c_played) == 0)  # tried playing but failed
       ) or postaction==DISCARD:  
        score += -0.2
        
    